In [1]:
import pandas as pd
import os
import time

# กำหนดที่อยู่ของโฟลเดอร์หลัก
main_folder = r'P:\Support Data\ICD-CMI, OR, COVID, Dead\ICD, CMI\2022-2024\DATA CMI\RAW DATA'

# สร้างรายการเก็บข้อมูลจากไฟล์
dataframes = []

# วัดเวลาเริ่มต้น
start_time = time.time()

# วนลูปทุกโฟลเดอร์และไฟล์ในโฟลเดอร์หลัก
for root, dirs, files in os.walk(main_folder):
    for file_name in files:
        # ตรวจสอบนามสกุลของไฟล์
        if file_name.endswith(('.xlsx', '.xls', '.csv', '.ods')):
            # สร้างทางเข้าสำหรับไฟล์
            file_path = os.path.join(root, file_name)
            
            # ตรวจสอบนามสกุลของไฟล์
            if file_name.endswith(('.xlsx', '.xls')):
                # ใช้ ExcelFile ในการอ่านข้อมูลจาก Excel ที่มีหลาย sheet
                xls = pd.ExcelFile(file_path)
                for sheet_name in xls.sheet_names:
                    df = pd.read_excel(xls, sheet_name)
                    # เพิ่มคอลัมน์ 'Source' เพื่อระบุที่มาของข้อมูล
                    df['Source'] = os.path.join(root, file_name)
                    dataframes.append(df)
            elif file_name.endswith('.csv'):
                df = pd.read_csv(file_path)
                # เพิ่มคอลัมน์ 'Source' เพื่อระบุที่มาของข้อมูล
                df['Source'] = os.path.join(root, file_name)
                dataframes.append(df)
            elif file_name.endswith('.ods'):
                df = pd.read_excel(file_path, engine='odf')
                # เพิ่มคอลัมน์ 'Source' เพื่อระบุที่มาของข้อมูล
                df['Source'] = os.path.join(root, file_name)
                dataframes.append(df)

# วัดเวลาสิ้นสุด
elapsed_time = time.time() - start_time

# ต่อข้อมูลจาก DataFrame ทั้งหมด
merged_data = pd.concat(dataframes, ignore_index=True)

# แสดงผลลัพธ์
print("Merged Data:")
print(merged_data)

# แสดงเวลาที่ใช้
print("\nElapsed Time:", elapsed_time, "seconds")


Merged Data:
        HCODE              AN            HN     DATEADM TIMEADM     DATEDSC  \
0        40.0   I40-24-006908  40-24-062963  29/11/2024   23:52  30/11/2024   
1        40.0   I40-24-006887  40-24-033947  28/11/2024   17:49  29/11/2024   
2        40.0   I40-24-006815  40-24-062348  24/11/2024   23:01  25/11/2024   
3        40.0   I40-24-006600  40-24-007795  15/11/2024   15:09  16/11/2024   
4        40.0   I40-24-006609  40-24-060401  15/11/2024   22:13  16/11/2024   
...       ...             ...           ...         ...     ...         ...   
133727    NaN             NaN           NaN         NaN     NaN         NaN   
133728    NaN             NaN           NaN         NaN     NaN         NaN   
133729    NaN  Total case IPD            41         NaN     NaN         NaN   
133730    NaN          Adj RW          36.9         NaN     NaN         NaN   
133731    NaN             CMI           0.9         NaN     NaN         NaN   

       TIMEDSC  CALLOS  LEAVEDAY  ACTL

In [2]:
merged_data = merged_data.dropna(subset=['HCODE'])

In [3]:
selected_columns_merged_data = merged_data[['HCODE', 'AN','HN','DATEADM','TIMEADM','DATEDSC','TIMEDSC','CALLOS','ACTLOS','WARDDSC','DEPT','AGE','AGEDAY','SEX','DISCHT','ADMWT',
                                            'PDX', 'SDX1', 'SDX2', 'SDX3', 'SDX4', 'SDX5', 'SDX6', 'SDX7', 'SDX8', 'SDX9', 'SDX10', 'SDX11', 'SDX12',
                                              'PROC1', 'PROC2', 'PROC3', 'PROC4', 'PROC5', 'PROC6', 'PROC7', 'PROC8', 'PROC9', 'PROC10',
                                             'PROC11', 'PROC12', 'PROC13', 'PROC14', 'PROC15', 'PROC16', 'PROC17', 'PROC18', 'PROC19', 'PROC20',
                                            'MDC','DRG','RW','ADJRW','WTLOS','Source']]

In [4]:
# สร้างฟังก์ชันสำหรับการแปลงสูตร Excel
def cal_OR_NON_OR_DRG(value):
    if 0 <= int(value[3:5]) <= 49:
        return 'OR'
    elif 50 <= int(value[3:5]) <= 99:
        return 'NON OR'
    else:
        return 'N/A'


# สร้าง column ใหม่โดยใช้ apply
selected_columns_merged_data.loc[:, 'OR TYPE'] = selected_columns_merged_data['DRG'].apply(cal_OR_NON_OR_DRG)

TypeError: 'int' object is not subscriptable

In [ ]:
selected_columns_merged_data

In [ ]:
# ฟังก์ชันสำหรับการแปลงวันที่
def convert_date_format(date_str):
    try:
        return pd.to_datetime(date_str, format='%d-%m-%Y')
    except ValueError:
        return pd.to_datetime(date_str, format='%d/%m/%Y', errors='coerce')

selected_columns_merged_data['DATEADM'] = selected_columns_merged_data['DATEADM'].apply(convert_date_format)
selected_columns_merged_data['DATEDSC'] = selected_columns_merged_data['DATEDSC'].apply(convert_date_format)

In [ ]:
selected_columns_merged_data

In [ ]:
def replace_icd_10(df, columns):
    for col in columns:
        # ตรวจสอบและจัดการค่า NaN ในคอลัมน์
        df[col].fillna('', inplace=True)

        # แปลงค่าในคอลัมน์
        df[col] = df[col].str.strip("'").apply(lambda x: x[:3] + ('.' + x[3:] if len(x) > 3 else ''))

    return df

In [ ]:
# เรียกใช้งานฟังก์ชัน
columns_to_process = ['PDX', 'SDX1', 'SDX2', 'SDX3', 'SDX4', 'SDX5', 'SDX6', 'SDX7', 'SDX8', 'SDX9', 'SDX10', 'SDX11', 'SDX12']
selected_columns_merged_data = replace_icd_10(selected_columns_merged_data, columns_to_process)

In [ ]:
def replace_icd_9(df, columns):
    for col in columns:
        # ตรวจสอบและจัดการค่า NaN ในคอลัมน์
        df[col].fillna('', inplace=True)

         # แปลงค่าในคอลัมน์
        df[col] = df[col].str.strip("'")

    return df

In [ ]:
# เรียกใช้งานฟังก์ชัน
columns_to_process = ['PROC1', 'PROC2', 'PROC3', 'PROC4', 'PROC5', 'PROC6', 'PROC7', 'PROC8', 'PROC9', 'PROC10',
                     'PROC11', 'PROC12', 'PROC13', 'PROC14', 'PROC15', 'PROC16', 'PROC17', 'PROC18', 'PROC19', 'PROC20']
selected_columns_merged_data = replace_icd_9(selected_columns_merged_data, columns_to_process)

In [ ]:
def replace_symbol(df, columns):
    for col in columns:
        # ตรวจสอบและจัดการค่า NaN ในคอลัมน์
        df[col].fillna('', inplace=True)

        # แปลงค่าในคอลัมน์
        df[col] = df[col].str.strip("'")

    return df

In [ ]:
# เรียกใช้งานฟังก์ชัน
columns_to_process = ['MDC', 'DRG']
selected_columns_merged_data = replace_symbol(selected_columns_merged_data, columns_to_process)

In [ ]:
selected_columns_merged_data

In [ ]:
selected_columns_merged_data.info()

In [ ]:
selected_columns_merged_data_no_duplicates = selected_columns_merged_data.drop_duplicates(subset='AN')

In [ ]:
selected_columns_merged_data_no_duplicates.info()

In [ ]:
import time

excel_file_path = 'RAW_CMI.xlsx'  # ระบุ path และชื่อไฟล์ Excel ที่คุณต้องการ

# บันทึกเวลาเริ่มต้น
start_time = time.time()

# Export DataFrame เป็นไฟล์ Excel ด้วย Pandas
selected_columns_merged_data.to_excel(excel_file_path, index=False)

# คำนวณและแสดงเวลาที่ใช้
elapsed_time = time.time() - start_time
print(f'เวลาที่ใช้ในการ export เป็นไฟล์ Excel: {elapsed_time:.2f} วินาที')